In [ ]:
import numpy as np
import random

def Score(motifs):
    """
    Computes the score of a set of motifs. 
    The score is the sum of the number of non-consensus nucleotides 
    at each position across the motifs.
    
    Parameters:
        motifs (list of str): A list of DNA strings (motifs) of equal length.
    
    Returns:
        int: The total score (lower is better).
    """
    score = 0
    k = len(motifs[0])
    counts = np.zeros((4, k))  # A, C, G, T rows

    for i in range(k):
        column = [motif[i] for motif in motifs]
        counts[0][i] = column.count('A')
        counts[1][i] = column.count('C')
        counts[2][i] = column.count('G')
        counts[3][i] = column.count('T')
        
        max_count = max(counts[:, i])  # Count of most common base at this position
        score += sum(counts[:, i]) - max_count  # Non-consensus nucleotides

    return int(score)

def form_profile(motifs):
    """
    Generates a profile matrix with Laplace smoothing (adds pseudocounts).
    
    Parameters:
        motifs (list of str): A list of DNA motifs.
    
    Returns:
        dict: A profile dictionary where keys are 0–3 (for A, C, G, T) and 
              values are lists of probabilities at each position.
    """
    k = len(motifs[0])
    t = len(motifs)
    profile = {i: [0] * k for i in range(4)}  # A:0, C:1, G:2, T:3

    for i in range(k):
        column = [motif[i] for motif in motifs]
        for j, nucleotide in enumerate('ACGT'):
            # Apply Laplace pseudocount (+1)
            profile[j][i] = (column.count(nucleotide) + 1) / (t + 4)
    return profile

def Score_brute(profile, kmer):
    """
    Calculates the probability score of a k-mer given a profile matrix.
    
    Parameters:
        profile (dict): Profile matrix with nucleotide probabilities.
        kmer (str): A DNA substring.
    
    Returns:
        float: The probability score of the k-mer.
    """
    score = 1
    for ind, nucleo in enumerate(kmer):
        score *= profile['ACGT'.index(nucleo)][ind]
    return score

def form_motifs(profile, dna, k):
    """
    Given a profile and a list of DNA strings, returns the most probable 
    k-mer from each string.
    
    Parameters:
        profile (dict): Profile matrix.
        dna (list of str): List of DNA strings.
        k (int): Length of motifs.
    
    Returns:
        list of str: Most probable k-mers from each DNA string.
    """
    motifs = []
    for i in range(len(dna)):
        best_kmer = ''
        best_score = float('-inf')
        for j in range(len(dna[i]) - k + 1):
            kmer = dna[i][j:j+k]
            score = Score_brute(profile, kmer)
            if score > best_score:
                best_score = score
                best_kmer = kmer
        motifs.append(best_kmer)
    return motifs

def RandomizedMotifSearch(dna, k, t):
    """
    Performs one run of the randomized motif search algorithm.
    
    Parameters:
        dna (list of str): DNA sequences.
        k (int): Length of the motif.
        t (int): Number of DNA strings.
    
    Returns:
        list of str: The best motifs found in this run.
    """
    # Step 1: Randomly select initial motifs
    motifs = []
    for i in range(t):
        start_index = random.randint(0, len(dna[i]) - k)
        motifs.append(dna[i][start_index:start_index + k])
    
    best_motifs = motifs

    # Iteratively refine motifs until local optimum
    while True:
        profile = form_profile(motifs)
        motifs = form_motifs(profile, dna, k)
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
        else:
            return best_motifs

def RandomizedSearch(dna, k, t):
    """
    Repeats the randomized motif search multiple times to avoid local minima.
    
    Parameters:
        dna (list of str): DNA sequences.
        k (int): Motif length.
        t (int): Number of DNA strings.
    
    Returns:
        list of str: The best motifs found over all runs.
    """
    best_motifs = RandomizedMotifSearch(dna, k, t)
    for _ in range(1000):
        motifs = RandomizedMotifSearch(dna, k, t)
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
    return best_motifs


In [7]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA' ,'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC', 'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
k = 8
t = 5
RandomizedSearch(dna, k, t)

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

In [9]:
with open('dataset_30307_5.txt','r') as file:
    first_line = file.readline().strip().split()
    k, t = map(int,first_line)
    dna = file.read().strip().split()
print(' '.join(RandomizedSearch(dna, k , t)))

AAAGTTCACAAACTT TAATCCTACCATAGG TAAGTTTTGTATAGG TAAGTTTACCCATGG TAAGTTCGTCATAGG ATAGTTTACCATAGT TTTTTTTACCATAGG GAAGTTTACCATACC TAAGTGCTCCATAGG TAAGTTATGCATAGG TAAGCAGACCATAGG CCTGTTTACCATAGG TACCATTACCATAGG TAAGTTTATAGTAGG TAAGTTTACCATTAT TAAGTTTACATAAGG TAAGTAGCCCATAGG TAATCATACCATAGG TAAGTTTACCACCTG TAAGGCCACCATAGG


In [ ]:
import random

def Score_brute(profile, kmer):
    """
    Calculates the probability score of a k-mer given a profile matrix.
    
    Parameters:
        profile (dict): Profile matrix mapping ACGT to probabilities.
        kmer (str): A DNA substring of length k.
    
    Returns:
        float: The product of probabilities from the profile for this k-mer.
    """
    score = 1
    for ind, nucleo in enumerate(kmer):
        score *= profile['ACGT'.index(nucleo)][ind]
    return score

def prob_distribution(profile, kmers):
    """
    Converts raw profile scores into a normalized probability distribution 
    for all possible k-mers from a DNA string.
    
    Parameters:
        profile (dict): Profile matrix.
        kmers (list of str): All possible k-mers from one DNA sequence.
    
    Returns:
        dict: A probability distribution over the k-mers.
    """
    prob_dict = {kmer: Score_brute(profile, kmer) for kmer in kmers}
    total = sum(prob_dict.values())
    prob_dist = {kmer: prob_dict[kmer] / total for kmer in kmers}
    return prob_dist

def random_kmer(prob_dist):
    """
    Samples a k-mer from a given probability distribution.
    
    Parameters:
        prob_dist (dict): Probability distribution over k-mers.
    
    Returns:
        str: A sampled k-mer.
    """
    kmers = list(prob_dist.keys())
    weights = list(prob_dist.values())
    return random.choices(kmers, weights=weights)[0]

def form_profile(motifs):
    """
    Builds a profile matrix from a list of motifs using Laplace smoothing.
    
    Parameters:
        motifs (list of str): DNA motifs of equal length.
    
    Returns:
        dict: Profile matrix with nucleotide probabilities.
    """
    k = len(motifs[0])
    t = len(motifs)
    profile = {i: [0] * k for i in range(4)}  # A:0, C:1, G:2, T:3

    for i in range(k):
        column = [motif[i] for motif in motifs]
        for j, nucleotide in enumerate('ACGT'):
            # Laplace smoothing: add 1 to each count
            profile[j][i] = (column.count(nucleotide) + 1) / (t + 4)
    return profile

def Score(motifs):
    """
    Computes the total number of mismatches from the consensus across motifs.
    
    Parameters:
        motifs (list of str): DNA motifs.
    
    Returns:
        int: Score (lower is better).
    """
    k = len(motifs[0])
    counts = [[0] * k for _ in range(4)]  # A, C, G, T counts
    score = 0

    for i in range(k):
        for motif in motifs:
            counts['ACGT'.index(motif[i])][i] += 1
        max_count = max(counts[j][i] for j in range(4))
        score += len(motifs) - max_count
    return score

def gibbs_core(dna, k, t, N):
    """
    Core Gibbs sampling routine that runs for N iterations.
    
    Parameters:
        dna (list of str): DNA sequences.
        k (int): Motif length.
        t (int): Number of sequences.
        N (int): Number of iterations.
    
    Returns:
        tuple: (best motifs found, their score)
    """
    # Random initialization
    motifs = [
        dna[i][random.randint(0, len(dna[i]) - k):][:k]
        for i in range(t)
    ]
    best_motifs = motifs[:]
    best_score = Score(best_motifs)

    for _ in range(N):
        i = random.randrange(t)
        subset_motifs = motifs[:i] + motifs[i + 1:]  # Exclude one motif
        profile = form_profile(subset_motifs)

        # Generate all k-mers for sequence i
        kmers = [dna[i][j:j + k] for j in range(len(dna[i]) - k + 1)]
        prob_dist = prob_distribution(profile, kmers)
        new_motif = random_kmer(prob_dist)
        motifs[i] = new_motif

        current_score = Score(motifs)
        if current_score < best_score:
            best_motifs = motifs[:]
            best_score = current_score

    return best_motifs, best_score

def gibbs_sampling(dna, k, t, N):
    """
    Runs Gibbs sampling multiple times to escape local minima and find
    the best motif set.
    
    Parameters:
        dna (list of str): List of DNA sequences.
        k (int): Length of the motif.
        t (int): Number of DNA strings.
        N (int): Number of iterations per run.
    
    Returns:
        tuple: Best motifs and their score found across all runs.
    """
    best_motifs, best_score = gibbs_core(dna, k, t, N)
    for _ in range(120):
        motifs, score = gibbs_core(dna, k, t, N)
        if score < best_score:
            best_motifs, best_score = motifs, score
    return best_motifs, best_score


In [90]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCT']
k = 8
t = 5
N = 100
gibbs_sampling(dna, k, t, N)

Motifs: ['CTCTCGGG', 'GTGTAAGT', 'TACCGAGA', 'GTCGGTGA', 'ATGTTGGC']
Randomly selected index: 2
Updated motifs: ['CTCTCGGG', 'GTGTAAGT', 'TATACAGG', 'GTCGGTGA', 'ATGTTGGC']
Best motifs: ['CTCTCGGG', 'GTGTAAGT', 'TATACAGG', 'GTCGGTGA', 'ATGTTGGC']
Motifs: ['CTCTCGGG', 'GTGTAAGT', 'TATACAGG', 'GTCGGTGA', 'ATGTTGGC']
Randomly selected index: 1
Updated motifs: ['CTCTCGGG', 'GTATGTGT', 'TATACAGG', 'GTCGGTGA', 'ATGTTGGC']
Motifs: ['CTCTCGGG', 'GTATGTGT', 'TATACAGG', 'GTCGGTGA', 'ATGTTGGC']
Randomly selected index: 2
Updated motifs: ['CTCTCGGG', 'GTATGTGT', 'ATACAGGC', 'GTCGGTGA', 'ATGTTGGC']
Best motifs: ['CTCTCGGG', 'GTATGTGT', 'ATACAGGC', 'GTCGGTGA', 'ATGTTGGC']
Motifs: ['CTCTCGGG', 'GTATGTGT', 'ATACAGGC', 'GTCGGTGA', 'ATGTTGGC']
Randomly selected index: 0
Updated motifs: ['GTTCAGTA', 'GTATGTGT', 'ATACAGGC', 'GTCGGTGA', 'ATGTTGGC']
Motifs: ['GTTCAGTA', 'GTATGTGT', 'ATACAGGC', 'GTCGGTGA', 'ATGTTGGC']
Randomly selected index: 2
Updated motifs: ['GTTCAGTA', 'GTATGTGT', 'GTATACAG', 'GTCGGTGA',

(['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG'], 11.0)

In [1]:
import random
import numpy as np

def Score_brute(profile, kmer):
    """
    Computes the probability of a k-mer given a profile matrix.
    Multiplies the corresponding probabilities from the profile for each nucleotide.

    Args:
        profile (dict): A profile matrix with 4 rows (A, C, G, T) and k columns.
        kmer (str): A DNA k-mer.

    Returns:
        float: The probability score of the k-mer.
    """
    score = 1
    for ind, nucleo in enumerate(kmer):
        score *= profile['ACGT'.index(nucleo)][ind]
    return score

def prob_distribution(profile, kmers):
    """
    Computes a normalized probability distribution over all k-mers based on a profile matrix.

    Args:
        profile (dict): A profile matrix.
        kmers (list): List of k-mers from one sequence.

    Returns:
        dict: Mapping from each k-mer to its normalized probability.
    """
    prob_dict = {}
    prob_dist = {}
    for kmer in kmers:
        prob_dict[kmer] = Score_brute(profile, kmer)
    total = sum(prob_dict.values())
    for kmer in kmers:
        prob_dist[kmer] = prob_dict[kmer] / total
    return prob_dist

def random_kmer(prob_dist):
    """
    Samples a k-mer randomly from a given probability distribution.

    Args:
        prob_dist (dict): Dictionary of k-mer probabilities.

    Returns:
        str: A sampled k-mer.
    """
    kmers = []
    weights = []
    for key, value in prob_dist.items():
        kmers.append(key)
        weights.append(value)
    rand_kmer = random.choices(kmers, weights)[0]
    return rand_kmer

def form_profile(motifs):
    """
    Constructs a profile matrix from a list of motifs with pseudocounts.

    Args:
        motifs (list of str): List of k-mers (motifs).

    Returns:
        dict: Profile matrix as a dictionary mapping A, C, G, T to lists of probabilities.
    """
    k = len(motifs[0])
    t = len(motifs)
    profile = {i: [0] * k for i in range(len('ACGT'))}

    for i in range(k):
        column = [motif[i] for motif in motifs]
        for j, nucleotide in enumerate('ACGT'):
            profile[j][i] = (column.count(nucleotide) + 1) / (t + 4)  # pseudocounts
    return profile

def gibbs_core(dna, k, t, N):
    """
    Performs the core Gibbs Sampling procedure to refine motifs.

    Args:
        dna (list of str): Input DNA sequences.
        k (int): Length of motif.
        t (int): Number of sequences.
        N (int): Number of sampling iterations.

    Returns:
        tuple: (Best motif list found, corresponding score).
    """
    motifs = []
    for i in range(t):
        start_index = random.randint(0, len(dna[i]) - k)
        motifs.append(dna[i][start_index:start_index + k])

    best_motifs = motifs
    best_score = float('inf')

    for _ in range(N):
        i = random.randrange(t)
        subset_motifs = motifs[:i] + motifs[i + 1:]
        profile = form_profile(subset_motifs)
        kmers = [dna[i][j:j + k] for j in range(len(dna[i]) - k + 1)]
        prob_dist = prob_distribution(profile, kmers)
        new_motif = random_kmer(prob_dist)
        motifs[i] = new_motif

        current_score = Score(motifs)
        if current_score < best_score:
            best_motifs = motifs
            best_score = current_score
    return best_motifs, best_score

def gibbs_sampling(dna, k, t, N, iterations):
    """
    Runs the Gibbs sampling algorithm multiple times to avoid local optima.

    Args:
        dna (list of str): DNA sequences.
        k (int): Motif length.
        t (int): Number of sequences.
        N (int): Number of iterations for each sampling run.
        iterations (int): How many times to re-run the sampling.

    Returns:
        tuple: (Best motif list found, corresponding score).
    """
    best_motifs, best_score = gibbs_core(dna, k, t, N)
    for _ in range(iterations):
        motifs, current_score = gibbs_core(dna, k, t, N)
        if current_score < best_score:
            best_motifs = motifs
            best_score = current_score
    return best_motifs, best_score

def Score(motifs):
    """
    Computes the score of a motif alignment.
    The score is the total number of mismatches to the consensus string.

    Args:
        motifs (list of str): List of motifs.

    Returns:
        int: Total score (lower is better).
    """
    score = 0
    k = len(motifs[0])
    counts = np.zeros((4, k))

    for i in range(k):
        column = [motif[i] for motif in motifs]
        counts[0][i] = column.count('A')
        counts[1][i] = column.count('C')
        counts[2][i] = column.count('G')
        counts[3][i] = column.count('T')

        max_count = max(counts[:, i])
        score += sum(counts[:, i]) - max_count
    return score

def form_motifs(profile, dna, k):
    """
    Picks the highest scoring k-mer from each DNA sequence based on the profile.

    Args:
        profile (dict): Profile matrix.
        dna (list of str): DNA sequences.
        k (int): Motif length.

    Returns:
        list: Best-scoring k-mer from each sequence.
    """
    motifs = []
    for seq in dna:
        best_kmer = ''
        best_score = float('-inf')
        for j in range(len(seq) - k + 1):
            kmer = seq[j:j + k]
            score = Score_brute(profile, kmer)
            if score > best_score:
                best_score = score
                best_kmer = kmer
        motifs.append(best_kmer)
    return motifs

def read_dna_sequences(file_path):
    """
    Reads DNA sequences from a file. Assumes sequences are space- or newline-separated.

    Args:
        file_path (str): Path to file.

    Returns:
        list of str: DNA sequences.
    """
    with open(file_path, 'r') as file:
        dna = file.read().strip().split()
    return dna

def RandomizedSearch(file_path, k, t, num_iterations):
    """
    Runs the Randomized Motif Search algorithm, a simpler alternative to Gibbs sampling.

    Args:
        file_path (str): Path to input file.
        k (int): Motif length.
        t (int): Number of DNA sequences.
        num_iterations (int): How many random restarts to perform.

    Returns:
        tuple: (Best motif list found, corresponding score).
    """
    dna = read_dna_sequences(file_path)
    best_motifs = []
    best_score = float('inf')

    for _ in range(num_iterations):
        motifs = []
        for i in range(t):
            start_index = random.randint(0, len(dna[i]) - k)
            motifs.append(dna[i][start_index:start_index + k])

        while True:
            profile = form_profile(motifs)
            motifs = form_motifs(profile, dna, k)
            current_score = Score(motifs)
            if current_score < best_score:
                best_motifs = motifs
                best_score = current_score
            else:
                break

    return best_motifs, best_score

